# Design Information for the `composable` module

## Outline

1. Overview of curried functions and piping
2. A review of python call semantics
3. Transforming a regular python function to a curried function


### Expressing the type of a curried function.

While discussing curried functions, it is useful to express the type of such functions in a way that is similar to functional languages such as `Elm`

Consider the following function, which determines if `x > y` for any number.  

In [2]:
from typing import Union, List
from toolz import curry

Num = Union[int, float]

@curry
def gt(x: Num, y: Num) -> bool:
    return x > y

Since, `gt` is decorated with `curry`, we can create a partial function by providing only one argument

In [3]:
lt3 = gt(3)
lt3(2)

True

In [4]:
lt3(4)

False

So what is the type of the new function `lt3`? We can use the package `forge` to get the signature of this function. 

In [5]:
import forge

forge.repr_callable(lt3)

"gt(y: Union[int, float] = '__no__default__') -> bool"

This illustrates two things.

1. The type of partial application (through currying) has a clear relationship to the original type.
2. `forge` was able to get the signature mostly right.
2. <font color='red'> The toolz implementation of curry could use better signatures </font>

## Review of python parameters and call semantics

User defined Python functions allow for 4 types of function parameters.  There is soon to be a 5th, see [this PEP](https://www.python.org/dev/peps/pep-0570/); and for completeness we will include it here.  

Parameters can be

1. position-or-keyword parameters
2. position-only
3. keyword-only
4. var-positional
5. var-keyword

### Position-or-keyword arguments

Most parameters are position-or-keyword, which can be defined either with or without a default value.  In the following function, `x` and `y` are both position-or-keyword parameters, with `y` having a default value of `5`.

In [63]:
def f(x, y, z=5, a=1):
    return f"x == {x}, y == {y}, z == {z}, and a = {a}"

When defined with this type of parameter, parameters with defaults most follow non-default parameters 

In [64]:
def g(x, y = 5, z):
    pass

SyntaxError: non-default argument follows default argument (<ipython-input-64-b6f0c28b49b7>, line 1)

When defined using this type of parameter, arguments can be provided positionally or using keywords.  

First, we call `f` with 2, 3, and 4 positional arguments, respectively.

In [65]:
f(1,2)

'x == 1, y == 2, z == 5, and a = 1'

In [66]:
f(1, 2, 3)

'x == 1, y == 2, z == 3, and a = 1'

In [67]:
f(1, 2, 3, 4)

'x == 1, y == 2, z == 3, and a = 4'

Alternatively, we can call `f` using only keywords.

In [68]:
f(y = 1, x = 2)

'x == 2, y == 1, z == 5, and a = 1'

In [69]:
f(y = 1, x = 2, a = 3)

'x == 2, y == 1, z == 5, and a = 3'

In [70]:
f(y = 1, x = 2, a = 3, z = 4)

'x == 2, y == 1, z == 4, and a = 3'

Note that, when using keyword arguments, order is meaningless.

### Position only arguments

Most of the built-in functions in python use position-only arguments.  For example, consider the function `pow` from the `math` module.

In [71]:
from math import pow

help(pow)

Help on built-in function pow in module math:

pow(x, y, /)
    Return x**y (x to the power of y).



The `/` in the signature indicates that all preceding arguments are position-only and cannot be called as keywords.

For example, we can verify that `x` and `y` can be provided arguments positionally but not as keywords.

In [72]:
pow(2,3)

8.0

In [73]:
pow(x = 2, y = 3)

TypeError: pow() takes no keyword arguments

Currently, there is no way for users define positional-only arguments, but this change has been approved and will be implemented in a future version of python.

In [74]:
def my_pow(x, y, /): # Coming soon
    return x**y

SyntaxError: invalid syntax (<ipython-input-74-263422a67f62>, line 1)

### Keyword only arguments

We can also define parameters as keyword-only by including a single (unnamed) `*` before the list of keyword-only parameters

In [82]:
def g(x, y, *, z, a=1):
    return f"x == {x}, y == {y}, z == {z}, and a = {a}"

Note that keyword-only aprameters can be defined as required or with a default.  In this case, `z` is required and `a` has a default value of `1`.  Calling `g` without specifying `z` throws an exception.

In [86]:
g(1,2)

TypeError: g() missing 1 required keyword-only argument: 'z'

We can provide the required value of z as a keyword, but not positionally

In [88]:
g(1,2, z= 1)

'x == 1, y == 2, z == 1, and a = 1'

In [95]:
g(1, 2, 3)

TypeError: g() takes 2 positional arguments but 3 were given

As we have seen before, the order of keyword arguments does not matter.

In [96]:
g(1,2, a = 3, z= 4)

'x == 1, y == 2, z == 4, and a = 3'

### Var-parameters and Var-keywords

Python also allows semantics for a variable number of positional and keyword arguments.  To define a variable number of positional arguments is defined using the syntax `*name` and variable keyword parameters are defined using `**name`.

In [101]:
def h(*args, **kwargs):
    return f"args = {args} and kwargs = {kwargs}"

The values of the corresponding arguments are available in the body of the function as a `tuple` and `dict`
assigned the given names.

In [103]:
h(1,2,3, bob = 4, alice = 5)

"args = (1, 2, 3) and kwargs = {'bob': 4, 'alice': 5}"

## Thoughts about currying in Python

The complexity of python parameters does not match the simplicity of curried functions.  In pure languages like Haskell and Elm, all function parameters are effectively positional-only out of necessity: These statically-typed languages employ a type-system that cannot handle other types of parameters.

This leaves us with the task of defining currying in python in a meaningful and (hopefully) unambiguous manner.  Here are some of my thoughts.

#### Currying is all about position.

By definition, currying is a way of defining partial application of functions with positional parameters.


#### Keyword arguments in mathematical functions

The mathematical definition of a function does not allow keyword arguments and in python syntax would be defined with position-only parameters, i.e. $f(x, y, /) = x + y$. 

We can think of a function with a keyword argument as defining a class of functions, e.g. $f(x, /, *, y) = 2x + y$ defines a class of affine linear transformations with the specific function selected in the function call.  Similarly, a function with a keyword with a default such as $f(x, *, y=0) = 2x + y$ describes and one of these transformation, defaulting to $y=0$.

For example, in least squares regression can be characterized as selecting the optimal function from the class of functions defined by $f(x, /, *, a, b) = ax + b$.  When a statistician specifies such a model, they are not thinking of $a$ and $b$ in the same way that they think of $x$, which is captured by this representation.

#### Keyword arguments are orthogonal to positional arguments

The main point here is that we can think of keyword parameters as a different type of beast that this orthogonal to positional arguments.

#### Using position- and keyword-only parameters would be more pure

If we restrict ourselves to position- and keyword-only parameters, we get a more mathematically pure solution.

1. There is no interaction between the two types of parameters, which leads to orthogonality: currying is only about position and is not impacted by keywords.
2. There is a cleaner expression of the type of a function.


#### Using position-or-keyword arguments allows flexibility when piping

In particular, using curried functions in a pipe, allowing position-or-keyword arguments allows the added flexibility. To see this, consider the following funciton.

In [104]:
!pip install composable

     |████████████████████████████████| 51kB 927kB/s eta 0:00:01
  Created wheel for toolz: filename=toolz-0.10.0-cp37-none-any.whl size=54601 sha256=d0706b75b0a99209e910a9cf7ba3a5cc5a1de790fc75f08a760c744f310c2fb1
  Stored in directory: /Users/bn8210wy/Library/Caches/pip/wheels/e1/8b/65/3294e5b727440250bda09e8c0153b7ba19d328f661605cb151
Successfully built toolz
  Found existing installation: toolz 0.9.0
    Uninstalling toolz-0.9.0:
      Successfully uninstalled toolz-0.9.0


In [107]:
from composable import pipeable

@pipeable
def f(x, y):
    return x**y

We can naturally pipe a value into `y` by providing `x` positionally.

In [108]:
2 >> f(3)

9

Alternatively, we can pipe into `x` by providing the `y` arguments as a keyword.

In [111]:
2 >> f( y = 3)

8

#### Currying and default values DO NOT MIX!

To avoid ambiguity, we need to keep default values separate from currying.  To see this, consider the following function

In [113]:
def f(x, y=2):
    return x + y

If `f` is curried, what does `f(2)` mean?  It could be

1. A complete function application using the default value of `y`.
2. Partial application waiting for a value of `y`.

We need to avoid this ambiguity, and I would argue that 1. is the more natural solution.  **This means that we will need to remove the ability to provide default parameters positionally**.

#### Currying and var-positional parameters DO NOT MIX

Similar to the problem we saw above, including var-positional parameters in a curried function leads to ambiguity. In fact it is even worse, as it is impossible to determine when we have a complete application of the function.  To see this consider 

```
def f(*args):
    return args
```

So what is `f(1,2)`?  Is this a partial application waiting for additional arguments?  Is this a complete application?  There is literally no way to tell.  **This means that we will need to remove var-positional arguments and add this functionality another way.** 

An obvious solution is to convert the var-position parameter to a keyword-only parameter that defaults to the empty `tuple`.  This solution will

1. Allow an unambiguous way to specify a variable number of extra arguments, and
2. Still work with the existing function definition, as it will supply the same data structure with the same name.

#### A var-keyword is just fine.

In the same way that keyword arguments with a given default are orthogonal to currying, we can leave a var-keyword argument alone.

## The MAIN question.

Do we

1. go for purity at the expense of flexibility with position- and keyword-only arguments, or
2. allow for added flexibility using 

## Options

By necessity, we both solutions will require

1. Converting `*args` to `args = ()`
2. Converting default position-or-keyword parameters to keyword-only parameters.

Then we have a choice for dealing with non-default position-or-keyword parameters.

3. **Pure solution:** Convert all non-default position-or-keyword parameters to position-only
4. **Flexible solution:** Convert all non-default position-only parameters to position-or-keyword

In [114]:
import forge

In [121]:
isPosOnly = lambda t: t == forge.FParameter.POSITIONAL_ONLY

In [124]:
my_pow = forge.copy(pow)(pow)
my_pow

<function math.pow(x, y, /)>

In [129]:
forge.repr_callable(my_pow)

'pow(x, y, /)'

In [518]:
dir(my_pow)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__mapper__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__signature__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']

In [225]:
type(my_pow)

function

In [224]:
my_pow.__mapper__

<Mapper (x, y, /) => (x, y, /)>

In [175]:
def makePOK(param):
    if param.POSITIONAL_ONLY == param.kind:
        return param.replace(kind = param.POSITIONAL_OR_KEYWORD)
    else:
        return param

In [176]:
def makeAllPOK(previous):
    return previous.replace(
                        parameters = map(makePOK, previous), 
                        __validate_parameters__=False,
                        )

forge.manage(makeAllPOK)(my_pow)

<function math.pow(x, y)>

In [307]:
import forge

def inspect(previous):
    #print([dir(a) for a in previous])
    print("\n\ndir\n\n", dir(previous[0]))
    print("\n\nparam\n\n", [a for a in previous])
    print("\n\nkind\n\n", [a.kind for a in previous])
    print("\n\nbound\n\n", [a.bound for a in previous])
    print("\n\ndefault\n\n", [a.default for a in previous])
    print("\n\nno default\n\n", [a.default is forge.empty for a in previous])
    return previous
#     return previous.replace(
#         parameters=previous[::-1],
#         __validate_parameters__=False,
#    )

@forge.manage(inspect)
def func(a, b, c, *args, d = 2, n = None):
    pass

# assert forge.repr_callable(func) == 'func(c, b, a)'



dir

 ['KEYWORD_ONLY', 'POSITIONAL_ONLY', 'POSITIONAL_OR_KEYWORD', 'VAR_KEYWORD', 'VAR_POSITIONAL', '__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_creation_counter', '_creation_order', 'apply_conversion', 'apply_default', 'apply_validation', 'bound', 'contextual', 'converter', 'create_contextual', 'create_keyword_only', 'create_positional_only', 'create_positional_or_keyword', 'create_var_keyword', 'create_var_positional', 'default', 'empty', 'from_native', 'interface_name', 'kind', 'metadata', 'name', 'native', 'replace', 'type', 'validator']


param

 [<FParameter "a">, <FParameter "b">, <FParameter "c">, <FParameter "*args">, <FParameter "d=2">, <FParameter "n=None">]


kind

In [416]:
@forge.modify('a', kind = forge.FParameter.POSITIONAL_ONLY)
def f(a, x, y, b = 5, *c, d, e = 5, m, **k):
    return f"{a} {x} {y} {b} {c} {d} {m} {k}" 
f(1, 3, 5, 6, 7, 8, 9, 10, d= 7, m = None)

'1 3 5 6 (7, 8, 9, 10) 7 None {}'

In [229]:
forge.manage(inspect)(f)

[<FParameter "a">, <FParameter "x">, <FParameter "y">, <FParameter "b=5">, <FParameter "*c">, <FParameter "d">, <FParameter "e=5">, <FParameter "m">, <FParameter "**k">]
[<_ParameterKind.POSITIONAL_ONLY: 0>, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, <_ParameterKind.VAR_POSITIONAL: 2>, <_ParameterKind.KEYWORD_ONLY: 3>, <_ParameterKind.KEYWORD_ONLY: 3>, <_ParameterKind.KEYWORD_ONLY: 3>, <_ParameterKind.VAR_KEYWORD: 4>]
[False, False, False, False, False, False, False, False, False]
[<empty>, <empty>, <empty>, 5, <empty>, <empty>, 5, <empty>, <empty>]
[True, True, True, False, True, True, False, True, True]


<function __main__.f(a, /, x, y, b=5, *c, d, e=5, m, **k)>

In [336]:
def isPositionalOnly(param):
    return param.kind == param.POSITIONAL_ONLY

def isPositionalOrKeyword(param):
    return param.kind == param.POSITIONAL_OR_KEYWORD

def isPositional(param):
    return isPositionalOnly(param) or isPositionalOrKeyword(param)

def notPositional(param):
    return isKeywordOnly(param) or isVarKeyword(param)

def isKeywordOnly(param):
    return param.kind == param.KEYWORD_ONLY

def isVarKeyword(param):
    return param.kind == param.VAR_KEYWORD

def isVarPositional(param):
    return param.kind == param.VAR_POSITIONAL

def hasDefault(param):
    return param.default is not forge.empty

def noDefault(param):
    return param.default is forge.empty

In [282]:
help(forge.findparam)

Help on function findparam in module forge._signature:

findparam(parameters: Iterable[~_T_PARAM], selector: Union[str, Iterable[str], Callable[[~_T_PARAM], bool]]) -> Iterator[~_T_PARAM]
    Return an iterator yielding those parameters (of type
    :class:`inspect.Parameter` or :class:`~forge.FParameter`) that are
    mached by the selector.
    
    :paramref:`~forge.findparam.selector` is used differently based on what is
    supplied:
    
    - str: a parameter is found if its :attr:`name` attribute is contained
    - Iterable[str]: a parameter is found if its :attr:`name` attribute is
        contained
    - callable: a parameter is found if the callable (which receives the
        parameter), returns a truthy value.
    
    :param parameters: an iterable of :class:`inspect.Parameter` or
        :class:`~forge.FParameter`
    :param selector: an identifier which is used to determine whether a
        parameter matches.
    :returns: an iterator yield parameters



In [505]:
[(p, isPositional(p)) for p in forge.findparam(forge.FSignature.from_callable(f), lambda x: True)]

[(<FParameter "a">, True),
 (<FParameter "x">, True),
 (<FParameter "y">, True),
 (<FParameter "b=5">, True),
 (<FParameter "*c">, False),
 (<FParameter "d">, False),
 (<FParameter "e=5">, False),
 (<FParameter "m">, False),
 (<FParameter "**k">, False)]

In [525]:
def curryTransformParam(param):
    if isPositionalOnly(param):
        if param.default is not forge.empty:
            raise TypeError(f"Curried functions cannot contain positional-only parameters with a default")
        return param.replace(kind = param.POSITIONAL_OR_KEYWORD)
#     elif isPositionalOrKeyword(param) and hasDefault(param):
#         return param.replace(kind = param.KEYWORD_ONLY)
#     elif isVarPositional(param):
#         return param.replace(name="_star_" + param.name, kind = param.KEYWORD_ONLY, default = tuple([]))
    else:
        return param

In [526]:
def curryTransformSign(previous):
    return previous.replace(
                        parameters = map(curryTransformParam, previous), 
                        __validate_parameters__=False,
                        )

In [527]:
curryTransform = forge.manage(curryTransformSign)

In [528]:
forge.repr_callable(f)

'f(a, /, x, y, b=5, *c, d, e=5, m, **k)'

In [529]:
fc = curryTransform(f)
forge.repr_callable(fc)

'f(a, x, y, b=5, *c, d, e=5, m, **k)'

In [530]:
fcSign = forge.FSignature.from_callable(fc)
fcSign

<FSignature (a, x, y, b=5, *c, d, e=5, m, **k)>

In [531]:
[p for p in forge.findparam(fcSign, lambda p: True)]

[<FParameter "a">,
 <FParameter "x">,
 <FParameter "y">,
 <FParameter "b=5">,
 <FParameter "*c">,
 <FParameter "d">,
 <FParameter "e=5">,
 <FParameter "m">,
 <FParameter "**k">]

## <font color="blue"> RULE: When binding a value to a positional parameter, move the result to position only </font>

In [532]:
fc1 = forge.modify( 'a'
                   , default=2
                   , bound=True
                   , kind=forge.FParameter.POSITIONAL_ONLY,
    )(fc)

In [533]:
fc1

<function __main__.f(x, y, b=5, *c, d, e=5, m, **k)>

In [534]:
forge.repr_callable(fc1)

'f(x, y, b=5, *c, d, e=5, m, **k)'

In [535]:
fcSign1 = forge.FSignature.from_callable(fc1)
fcSign1

<FSignature (x, y, b=5, *c, d, e=5, m, **k)>

In [536]:
[p for p in forge.findparam(fcSign1, lambda p: True)]

[<FParameter "x">,
 <FParameter "y">,
 <FParameter "b=5">,
 <FParameter "*c">,
 <FParameter "d">,
 <FParameter "e=5">,
 <FParameter "m">,
 <FParameter "**k">]

In [537]:
fc

<function __main__.f(a, x, y, b=5, *c, d, e=5, m, **k)>

In [538]:
fc2 = forge.modify( 'x'
                   , default=1
                   , bound=True
                   , kind=forge.FParameter.POSITIONAL_ONLY,
    )(fc1)

In [539]:
def bind_positional(name, value):
    """ Creates a Revision decorator that will bind value to name.
    
    This is intended to be applyied to positional-or-keyword params and
    will changing the location to position-only to avoid an invalid signature.
    """
    return forge.modify( name
                       , default=value
                       , bound=True
                       , kind=forge.FParameter.POSITIONAL_ONLY,
                       )

In [580]:
def bind_positional(name, value):
    """ Creates a Revision decorator that will bind value to name and move it to the front.
    
    This is intended to be applyied to positional-or-keyword params and
    will changing the location to position-only to avoid an invalid signature.
    
    Note that the bound parameter is moved to position-only to insure a valid signature,
    which is required when binding a positional-or-keyword param with a keyword
    """
    return forge.compose(
            forge.modify( name
                       , default=value
                       , bound=True
                       , kind=forge.FParameter.POSITIONAL_ONLY,
                       ),
            forge.translocate(name, index = 0)
    )


In [565]:
fc2 = bind_positional('x', 2)(fc1)

In [566]:
fc2

<function __main__.f(y, b=5, *c, d, e=5, m, **k)>

In [567]:
forge.repr_callable(fc2)

'f(y, b=5, *c, d, e=5, m, **k)'

In [568]:
fcSign2 = forge.FSignature.from_callable(fc2)
fcSign2

<FSignature (y, b=5, *c, d, e=5, m, **k)>

In [570]:
[p for p in forge.findparam(fcSign2, lambda p: True)]

[<FParameter "y">,
 <FParameter "b=5">,
 <FParameter "*c">,
 <FParameter "d">,
 <FParameter "e=5">,
 <FParameter "m">,
 <FParameter "**k">]

In [573]:
fc_pos_by_kw = bind_positional('y', 5)(fc)

In [575]:
fc_pos_by_kw

<function __main__.f(a, x, b=5, *c, d, e=5, m, **k)>

In [578]:
sign_fc_pos_by_kw = forge.FSignature.from_callable(fc_pos_by_kw)
sign_fc_pos_by_kw

<FSignature (a, x, b=5, *c, d, e=5, m, **k)>

In [579]:
[p for p in forge.findparam(sign_fc_pos_by_kw, lambda p: True)]

[<FParameter "a">,
 <FParameter "x">,
 <FParameter "b=5">,
 <FParameter "*c">,
 <FParameter "d">,
 <FParameter "e=5">,
 <FParameter "m">,
 <FParameter "**k">]

## <font color="blue"> RULE: When binding a value to a keyword parameter, leave kind as keyword-only</font>

In [545]:
fc3 = forge.modify( 'd'
                   , default=3
                   , bound=True
                   #, kind=forge.FParameter.POSITIONAL_ONLY,
    )(fc2)

In [546]:
def bind_keyword(name, value):
    """ Creates a Revision decorator that will bind value to name, which is meant to be keyword-only.
    
    Note that applying this to other types of parameters might lead to invalid signatures.
    """
    return forge.modify( name
                       , default=value
                       , bound=True
                       )

In [547]:
fc3 = bind_keyword('d', 3)(fc2)

In [548]:
fc3

<function __main__.f(y, b=5, *c, e=5, m, **k)>

In [549]:
forge.repr_callable(fc3)

'f(y, b=5, *c, e=5, m, **k)'

In [550]:
fcSign3 = forge.FSignature.from_callable(fc3)
fcSign3

<FSignature (y, b=5, *c, e=5, m, **k)>

In [551]:
[p for p in forge.findparam(fcSign3, lambda p: True)]

[<FParameter "y">,
 <FParameter "b=5">,
 <FParameter "*c">,
 <FParameter "e=5">,
 <FParameter "m">,
 <FParameter "**k">]

In [588]:
def isBoundPO(param):
    return isPositionalOnly(param) and param.bound

def notBoundNoDefaultPOK(param):
    return isPositionalOrKeyword(param) and not param.bound and noDefault(param)

def validateCurried(params):
    return not all(isBoundPO(p) for p in params)

assert validateCurried(forge.findparam(fSign, lambda p: True))

In [589]:
fc

<function __main__.f(a, x, y, b=5, *c, d, e=5, m, **k)>

In [622]:
from itertools import chain

def position_or_name(i, param):
    """ Returns i if param is positional or param.name if keyword.
    """
    return (i, param.name) if isPositional(param) else (param.name, param.name)

def positional_names_and_revisions(sign, args):
    positional = tuple(forge.findparam(sign, isPositional))
    revisions = [bind_positional(p.name, v) for p, v in zip(positional, args)]
    names = {p.name for p in positional}
    return names, revisions

def var_positional_revisions(sign, num_pos, args):
    var_positional = tuple(forge.findparam(sign, isVarPositional))
    extra_args = args[num_pos:]
    var_arg_revisions = []
    if var_positional and extra_args:
        var_pos_name = var_positional[0]
        var_arg_revisions = [forge.modify(var_pos_name, default = extra_args, bound=True)]
    elif  extra_args:
        raise TypeError(f"{callable} takes {len(positional)} positional arguments but {len(args)} were given")
    return var_arg_revisions

def get_keyword_revisions(sign, pos_names, kwargs):
    keyword = {p.name:p for p in forge.findparam(sign, isKeywordOnly)}
    for n in kwargs.keys():
        if n not in keyword and n not in pos_names:
            raise  TypeError(f"{callable} got an unexpected keyword argument {n}")
    keyword_revisions = [bind_positional(n, v) if n in pos_names else bind_keyword(n, v) 
                         for n, v in kwargs.items()]
    return keyword_revisions
    
def bind_arguments(callable, args, kwargs):
    """ Returns a list of tuples uses to 
    
    Note that we assume that the callable adheres to the currying norms
    1. Bound positional-only, followed by
    2. keyword-only parameters, possibly including a transformed var-positional keyword.
    
    An exception will be called if callable does not follow these rules.
    """
    sign = forge.FSignature.from_callable(callable)
    
    if not validateCurried(sign):
        raise TypeError(f"{callable} does not have a valid curry signature")
        
    pos_names, pos_revisions = positional_names_and_revisions(sign, args)
    var_arg_revisions = var_positional_revisions(sign, len(pos_names), args)
    kw_revisions = get_keyword_revisions(sign, pos_names, kwargs)
    
    all_bindings = forge.compose(*chain(pos_revisions, kw_revisions, var_arg_revisions))
    return all_bindings(callable)

In [624]:
def incomplete_param(param):
    return (isPositional(param) or isKeywordOnly(param)) and noDefault(param)

def partial_application(callable):
    sign = forge.FSignature.from_callable(callable)
    return any(incomplete_param(p) for p in sign)
    

In [625]:
partial_application(fc)

True

In [626]:
partial_application(fc1)

True

In [627]:
partial_application(fc2)

True

In [628]:
partial_application(fc_new)

False

In [614]:
fc_new = bind_arguments(fc, [1,2], {'d': 5, 'm': 6, 'y': 3})

In [615]:
fc_new

<function __main__.f(b=5, *c, e=5, **k)>

In [616]:
fc_new()

'1 2 3 5 () 5 6 {}'

In [617]:
fc_new(bob = 12)

"1 2 3 5 () 5 6 {'bob': 12}"

In [618]:
fc_new(3)

'1 2 3 3 () 5 6 {}'

In [619]:
fc_new(3, bob = 12)

"1 2 3 3 () 5 6 {'bob': 12}"

In [620]:
fc_new(3,4,5)

'1 2 3 3 (4, 5) 5 6 {}'

In [621]:
fc_new(3,4,5, bob = 12)

"1 2 3 3 (4, 5) 5 6 {'bob': 12}"

In [476]:
forge.repr_callable(fc_new)

'f(y, *, b=5, c=(), e=5, **k)'

In [477]:
fcSign_new = forge.FSignature.from_callable(fc_new)
fcSign_new

<FSignature (y, *, b=5, c=(), e=5, **k)>

In [479]:
[p for p in forge.findparam(fcSign_new, lambda p: True)]

[<FParameter "y">,
 <FParameter "b=5">,
 <FParameter "c=()">,
 <FParameter "e=5">,
 <FParameter "**k">]

In [480]:
fc_new()

TypeError: f() missing a required argument: 'y'

In [482]:
fc_new(2)

"1 2 2 5 () 5 6 {'c': ()}"

In [485]:
fc_bound_kwargs = forge.modify('k', default={'kw': 12}, bound = True)(fc_new)

In [487]:
fc_bound_kwargs

<function __main__.f(y, *, b=5, c=(), e=5)>

In [488]:
forge.repr_callable(fc_bound_kwargs)

'f(y, *, b=5, c=(), e=5)'

In [491]:
sign_fc_bound_kwargs = forge.FSignature.from_callable(fc_bound_kwargs)
sign_fc_bound_kwargs

<FSignature (y, *, b=5, c=(), e=5)>

In [492]:
[p for p in forge.findparam(sign_fc_bound_kwargs, lambda p: True)]

[<FParameter "y">, <FParameter "b=5">, <FParameter "c=()">, <FParameter "e=5">]

In [493]:
fc_bound_kwargs(2)

"1 2 2 5 () 5 6 {'c': (), 'kw': 12}"

In [338]:
shortArgs = (1,2)

In [334]:
[isBoundPO(p) or notBoundNoDefaultPOK(p) or isKeywordOnly(p) for p in fSign]

[True, True, True, True, True, True, True, True, False]

In [329]:
shortArgs = (1,2)

In [323]:
completeArgs = (1, 2, 3)

In [324]:
longArgs = (1,2, 3, 4)

In [320]:
pos = [p for p in forge.findparam(fSign, isPositional)]
pos

[<FParameter "a">, <FParameter "x">, <FParameter "y">]

In [291]:
my_pow.__wrapped__

<function math.pow(x, y, /)>

In [128]:
my_new_pow = forge.modify(isPosOnly, kind=forge.FParameter.POSITIONAL_OR_KEYWORD)(my_pow)
my_new_pow

ValueError: No parameter matched selector '<function <lambda> at 0x1079ae840>'

#### A alternate way to express types of curried functions

In the same way that python uses `->` to indicate output type, `Elm` expresses the type of a multiple arity function using multiple `->`, one between each argument and a final one for the output. For example, we would express the type of `gt` as follows

In [6]:

@curry
def gt(x: Num, y: Num) -> bool:
    """ gt : Num -> Num -> bool
    """
    return x > y

The major of this representation is that it clearly expresses the types of partial application. 

For example, the type of `gt(3)`, we would remove the type of the first parameter, which is now bound, leaving `Num -> bool`.  

Similarly, the type of `gt(2,3)`, we would remove the first two types (which are both now bound) leaving just `bool`.

In [15]:
def f(x: Num, y: Num, z:Num = 2, a: Num = 0) -> Num:
    return (x + y)**z + a

In [22]:
forge.repr_callable(f)

'f(x: Union[int, float], y: Union[int, float], z: Union[int, float] = 2, a: Union[int, float] = 0) -> Union[int, float]'

#### Deficiency

While this expression of type is useful, it is not adequate to express the type of a python `callable`, due to 

1. Most positional arguments can be called as keywords.
2. Cannot express the type of varargs, i.e. `*args`
3. Leave no room for keyword arguments



## <font color='red'> TODO: So how do we express the type when there are keyword arguments

## Transforming a regular function to a curried functions

To transform a regular python function to a curried function in a we must deal with the following issues

1. Find a clean way to assign a value and return a new function
2. Dealing with <br>
    a. position-only and position-or-keyword arguments<br>
    b. variable arguments<br>
    c. keyword-only arguments<br>
    d. variable keyword arguments<br>
    
4. 

### Dealing with variable arguments

First, let's see how `toolz` handles curried functions with variable arguments.

In [29]:
@curry
def g(x, y, *args):
    return args

In [30]:
g(2)

<function g at 0x10e211378>

In [30]:
g(2)

<function g at 0x10e211378>

In [31]:
g(2, 3)

()

In [32]:
g(2, 3, 4)

(4,)

In [34]:
g(2, 3, 4, 5)

(4, 5)

So, only the position-or-keyword arguments are curried.  This means that the `*args` part needs to be completed in exactly one call.  This also means that providing values for `x` and `y` automatically sets args = ()

In [35]:
g(2)

<function g at 0x10e211378>

In [36]:
g(2)(3)

()

The only way to add extra arguments is adding them at the same time as y, which kind of violates the spirit of a curried function.

#### Possible solution - Change any vararg to an additional keyword argument.

One solution to this issue is convertion the `*args` to an optional keyword parameter named `args`.

So `g(1,2,3,4)` could be either `g(1)(2, args = (3,4))` or `g(1, args = (3,4))(2)`

#### Possible solution - add one more positional argument, which needs to be a list of args.

One solution to this issue is convertion the `*args` to an optional keyword parameter, named something like `_star_<name>` by default to avoid name conflicts.

#### Possible solution - Change any vararg to an additional keyword argument.

One solution to this issue is convertion the `*args` to an optional keyword parameter, named something like `_star_<name>` by default to avoid name conflicts.

So `g(1,2,3,4)` would be `g(1)(2)((3,4))`.

Note that using the same name is the only solution

Note that, if we change `*args` to position-or-keyword, this solution would also allow the previous syntax.

i.e. `g(1,2,3,4)` could be either `g(1)(2, args = (3,4)) == g(1, args = (3,4))(2) == g(1)(2)((3,4))`


In [39]:
@curry
def h(x, y, *args, z = None, **kwargs):
    pass 

In [40]:
forge.repr_callable(h)

"h(x='__no__default__', y='__no__default__', *args, z=None, **kwargs)"

In [397]:
f

<function __main__.f(a, /, x, y, b=5, *c, d, e=5, m, **k)>

In [400]:
(lambda x, y: None)(1,2,3)

TypeError: <lambda>() takes 2 positional arguments but 3 were given

In [401]:
bind_keyword('d', 3)(lambda : None)

ValueError: No parameter matched selector 'd'

In [402]:
from itertools import chain

In [404]:
f = lambda *args: args

In [405]:
f(*chain("abc", "def"))

('a', 'b', 'c', 'd', 'e', 'f')

In [558]:
t = tuple(range(5))
t

(0, 1, 2, 3, 4)

In [560]:
args = tuple(range(3))
args

(0, 1, 2)

In [561]:
t[len(args):]

(3, 4)

In [562]:
t[100:]

()

In [629]:
setattr(callable, '__rrshift__', lambda s, o: s(o))

AttributeError: 'builtin_function_or_method' object has no attribute '__rrshift__'

In [634]:
class Test:
    def __call__(self, x):
        return f"Hi {x}"

In [637]:
setattr(Test, '__rrshift__', lambda s, o: s(o))

In [638]:
f = Test()

In [639]:
2 >> f

'Hi 2'

In [644]:
f

In [645]:
f.__doc__ = "Test doc"

In [646]:
help(f)

Help on Test in module __main__ object:

class Test(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, x)
 |      Call self as a function.
 |  
 |  __rrshift__ lambda s, o
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [647]:
f.__doc__

'Test doc'

In [648]:
callable(f)

True

In [649]:
?callable

In [650]:
?help

In [651]:
forge.FSignature.from_callable(f)

<FSignature (x)>

In [652]:
f.__signature__ = forge.FSignature.from_callable(f)

In [653]:
help(f)

Help on Test in module __main__ object:

class Test(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, x)
 |      Call self as a function.
 |  
 |  __rrshift__ lambda s, o
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [654]:
bind_arguments.__doc__

' Returns a list of tuples uses to \n    \n    Note that we assume that the callable adheres to the currying norms\n    1. Bound positional-only, followed by\n    2. keyword-only parameters, possibly including a transformed var-positional keyword.\n    \n    An exception will be called if callable does not follow these rules.\n    '

In [655]:
type(bind_arguments)

function

In [656]:
help(bind_arguments)

Help on function bind_arguments in module __main__:

bind_arguments(callable, args, kwargs)
    Returns a list of tuples uses to 
    
    Note that we assume that the callable adheres to the currying norms
    1. Bound positional-only, followed by
    2. keyword-only parameters, possibly including a transformed var-positional keyword.
    
    An exception will be called if callable does not follow these rules.



In [657]:
bind_arguments.__module__

'__main__'

In [658]:
bind_arguments.__name__

'bind_arguments'

In [660]:
def help_header(aCallable):
    return f"Help on function {aCallable.__name__} in module {aCallable.__module__}:"

In [661]:
help_header(f)

AttributeError: 'Test' object has no attribute '__name__'